In [59]:
import numpy as np
import os.path
from datetime import date, timedelta
import json
import pandas as pd
matchNum = 1

In [60]:
def findDiffNumeric(metric, blue, red):
    total_metric_blue = sum([item[metric] for item in blue])
    total_metric_red = sum([item[metric] for item in red])
    metric_diff = total_metric_blue - total_metric_red
    return metric_diff

def getTotal(metric, data):
    total_metric = sum([item[metric] for item in data])    
    return total_metric

def findDiffBool(metric, team):
    metric_list = [item[metric] for item in team]
    metricTrueFalse = any(metric_list)
    return metricTrueFalse


In [61]:
f = open("./data/real_data/match_%d.json"%matchNum,"r")
data = json.load(f)

player_info = []
for item in data['participantIdentities']:
    playerid = int(item['participantId'])
    name = item['player']['summonerName']
    player_info.append((id,name))

df1 = pd.DataFrame(data['participants'])
df2 = pd.DataFrame(data['teams'])

#determine basic stats of winning team
victory_team = None
defeat_team = None
for i in range(2):
    df = df2.iloc[i]
    if df['winner']:
        victory_team = df2.iloc[[i]]
    else:
        defeat_team = df2.iloc[[i]]

#separate winners and losers
winners, losers = [],[]
for i in range(10):
    df = df1.iloc[i]
    playerId = df['participantId']
    item = df['stats']
    rank = df['highestAchievedSeasonTier']    
    kills = item['kills']
    deaths = item['deaths']
    assists = item['assists']
    gold = item['goldEarned']
    cs = item['minionsKilled'] + item['neutralMinionsKilledTeamJungle'] + item['neutralMinionsKilledEnemyJungle'] + item['neutralMinionsKilled']
    if item['winner']:
        item['ID'] = playerId
        item['rank'] = rank
        winners.append(item)
        #winners.append({'ID': playerId, 'rank': rank, 'kills': kills, 'deaths': deaths, 'assists': assists, 'gold':gold, 'creep_score':cs})
    else:
        item['ID'] = playerId
        item['rank'] = rank
        losers.append(item)
        #losers.append({'ID': playerId, 'rank': rank, 'kills': kills, 'deaths': deaths, 'assists': assists, 'gold':gold, 'creep_score':cs})

game_length = data['timeline']['frames'][-1]['timestamp']/1000/60 
victory_team.reset_index(inplace=True,drop=True)

if 'bans' in victory_team.columns:
    victory_team = victory_team.drop('bans',axis=1)
if 'bans' in defeat_team.columns:
        defeat_team = defeat_team.drop('bans',axis=1)

goldDiff = findDiffNumeric('goldEarned',winners,losers)
goldTotal = getTotal('goldEarned',winners)
killTotal = getTotal('kills',winners)    
killDiff = findDiffNumeric('kills',winners,losers)
towerDiff = victory_team.loc[0,'towerKills'] - defeat_team.loc[0,'towerKills']
baronDiff = victory_team.loc[0,'baronKills'] - defeat_team.loc[0,'baronKills']
dragonDiff = victory_team.loc[0,'dragonKills'] - defeat_team.loc[0,'dragonKills']
inhibitorDiff = victory_team.loc[0,'inhibitorKills'] - defeat_team.loc[0,'inhibitorKills']

victory_team.insert(victory_team.shape[1],'gameLength',game_length)
victory_team.insert(victory_team.shape[1],'goldTotal',goldTotal)
victory_team.insert(victory_team.shape[1],'goldDiff',goldDiff)
victory_team.insert(victory_team.shape[1],'killTotal',killTotal)
victory_team.insert(victory_team.shape[1],'killDiff',killDiff)
victory_team.insert(victory_team.shape[1],'towerDiff',towerDiff)
victory_team.insert(victory_team.shape[1],'inhibitorDiff',inhibitorDiff)
victory_team.insert(victory_team.shape[1],'baronDiff',baronDiff)
victory_team.insert(victory_team.shape[1],'dragonDiff',dragonDiff)

goldTotal = getTotal('goldEarned',losers)
killTotal = getTotal('kills',losers)
killDiff = findDiffNumeric('kills', losers, winners)
goldDiff = findDiffNumeric('goldEarned',losers,winners)    
towerDiff = defeat_team.loc[0,'towerKills'] - victory_team.loc[0,'towerKills']
baronDiff = defeat_team.loc[0,'baronKills'] - victory_team.loc[0,'baronKills']
dragonDiff = defeat_team.loc[0,'dragonKills'] - victory_team.loc[0,'dragonKills']
inhibitorDiff = defeat_team.loc[0,'inhibitorKills'] - victory_team.loc[0,'inhibitorKills']

defeat_team.insert(defeat_team.shape[1],'gameLength',game_length)
defeat_team.insert(defeat_team.shape[1],'goldTotal',goldTotal)
defeat_team.insert(defeat_team.shape[1],'goldDiff',goldDiff)
defeat_team.insert(defeat_team.shape[1],'killTotal',killTotal)
defeat_team.insert(defeat_team.shape[1],'killDiff',killDiff)
defeat_team.insert(defeat_team.shape[1],'towerDiff',towerDiff)
defeat_team.insert(defeat_team.shape[1],'inhibitorDiff',inhibitorDiff)
defeat_team.insert(defeat_team.shape[1],'baronDiff',baronDiff)
defeat_team.insert(defeat_team.shape[1],'dragonDiff',dragonDiff)    

        

In [40]:
match = victory_team.append(defeat_team)

In [41]:
match

,baronKills,dominionVictoryScore,dragonKills,firstBaron,firstBlood,firstDragon,firstInhibitor,firstRiftHerald,firstTower,inhibitorKills,...,winner,gameLength,goldTotal,goldDiff,killTotal,killDiff,towerDiff,inhibitorDiff,baronDiff,dragonDiff
0,1,0,2,True,False,True,False,True,True,0,...,True,23,44281,8632,32,15,4,0,1,2
0,0,0,0,False,True,False,False,False,False,0,...,False,23,35649,-8632,17,-15,-4,0,-1,-2


In [75]:
timeseries = []
df2 = pd.DataFrame(data['timeline']['frames'])
length = df2.shape[0]
for i in range(length):
    df = df2.iloc[i]
    time = df['timestamp']/1000/60
    wgold = 0
    for item in winners:
        ID = str(item['ID'])
        wgold += df['participantFrames'][ID]['totalGold']
        #cs = df['participantFrames'][ID]['minionsKilled']
        #jngcs = df['participantFrames'][ID]['jungleMinionsKilled']
        #wcreep_score += (cs + jngcs)
    lgold  = 0
    for item in losers:
        ID = str(item['ID'])
        lgold += df['participantFrames'][ID]['totalGold']
        #cs = df['participantFrames'][ID]['minionsKilled']
        #jngcs = df['participantFrames'][ID]['jungleMinionsKilled']
        #lcreep_score += (cs + jngcs)
    goldDiff = wgold-lgold
    timeseries.append((matchNum,time,wgold,lgold,goldDiff))
timeseries

[(1, 0, 2500, 2500, 0),
 (1, 1, 2500, 2500, 0),
 (1, 2, 2652, 2693, -41),
 (1, 3, 3856, 4907, -1051),
 (1, 4, 5434, 6173, -739),
 (1, 5, 7116, 7710, -594),
 (1, 6, 8270, 8889, -619),
 (1, 7, 10203, 10486, -283),
 (1, 8, 11364, 11630, -266),
 (1, 9, 13321, 13150, 171),
 (1, 10, 16343, 14149, 2194),
 (1, 11, 17716, 15505, 2211),
 (1, 12, 19859, 17301, 2558),
 (1, 13, 22265, 18220, 4045),
 (1, 14, 24753, 20893, 3860),
 (1, 15, 25857, 23343, 2514),
 (1, 16, 29519, 24485, 5034),
 (1, 17, 33090, 26937, 6153),
 (1, 18, 35781, 29019, 6762),
 (1, 19, 37223, 30207, 7016),
 (1, 20, 38754, 32343, 6411),
 (1, 21, 39803, 33896, 5907),
 (1, 22, 43802, 35387, 8415),
 (1, 23, 44281, 35649, 8632)]

In [79]:
match_time_df = pd.DataFrame(timeseries,columns=['matchId','time','vicGoldTotal','defGoldTotal','goldDiff'])
match_time_df.head(20)

,matchId,time,vicGoldTotal,defGoldTotal,goldDiff
0,1,0,2500,2500,0
1,1,1,2500,2500,0
2,1,2,2652,2693,-41
3,1,3,3856,4907,-1051
4,1,4,5434,6173,-739
5,1,5,7116,7710,-594
6,1,6,8270,8889,-619
7,1,7,10203,10486,-283
8,1,8,11364,11630,-266
9,1,9,13321,13150,171


In [68]:
df2.head()
    

,events,participantFrames,timestamp
0,NaN,"{u'10': {u'totalGold': 500, u'teamScore': 0, u...",33
1,"[{u'eventType': u'SKILL_LEVEL_UP', u'skillSlot...","{u'10': {u'totalGold': 500, u'teamScore': 0, u...",60048
2,"[{u'eventType': u'WARD_PLACED', u'creatorId': ...","{u'10': {u'totalGold': 502, u'teamScore': 0, u...",120056
3,"[{u'itemId': 2003, u'eventType': u'ITEM_DESTRO...","{u'10': {u'totalGold': 681, u'teamScore': 0, u...",180060
4,"[{u'eventType': u'WARD_PLACED', u'creatorId': ...","{u'10': {u'totalGold': 932, u'teamScore': 0, u...",240064


### 